# Using the World Bank API

With `Python`, we can easily access thousands of panel data series from the World Bank Indicators (WDI) .

The `pandas-datareader` library for the World Bank API integrates with `pandas`. It is not included in the Anaconda Python distribution, but it can easily be installed. From the command line in your shell, use this:

<center>`conda install pandas-datareader`</center>

First, we load the remote data access library for the World Bank Indicators API.

In [ ]:
from pandas_datareader import data, wb

## Searching for Indicators

The easiest way to identify an indicator is to search for it by name using a keyword or key phrase.

In [ ]:
wb.search('fertility rate')

We also can get a full list of indicators.

In [ ]:
indicators=wb.get_indicators()
indicators.head()

In [ ]:
indicators.info()

If we know the identifier  - or part of the identifier - for a particular indicator, we can look up details for it directly. Use the * character as a wildcard character.

In [ ]:
wb.search('gdp.*capita.*const')

In [ ]:
wb.search('gdp.*capita.*const').iloc[:,:2]

In [ ]:
wb.search('gdp.*capita.*const').name

## Identifying Countries

When retrieving a dataset, we can specifiy which country, countries or regions we want the data for. The locations are identified using the appropriate [ISO-2 code](http://en.wikipedia.org/wiki/ISO_3166-1_alpha-2). To look up countries we can download the full country list.

In [ ]:
countries=wb.get_countries()
countries.head()

In [ ]:
countries.info()

We can search for a particular country.

In [ ]:
countries[countries['name'] == 'Angola']

In [ ]:
countries[countries['iso3c'] == 'USA']

In [ ]:
countries[countries['iso3c'].str.contains('US')]

## Download Data for a Particular Indicator

Once we have identified one or more indicators for which we would like to download a dataset, we need to identify the year or range of years, and the country, countries or regions (identified via their ISO-2 code) for which we would like the data.

Then it is easy to download the dataset we want.

In [ ]:
df = wb.download(indicator='NY.GDP.PCAP.KD', country=['US', 'CA', 'MX'], start='2008', end=2018)
df

It looks like there isn't information yet for 2018.

In [ ]:
df = wb.download(indicator='NY.GDP.PCAP.KD', country=['US','CA','MX'], start='2008', end=2017)
df

The result is a properly formated `pandas` DataFrame with a hierarchical index, so it is easy to apply `groupby` transformations to it.

In [ ]:
df.groupby(level=0).mean()

In [ ]:
df['NY.GDP.PCAP.KD'].groupby(level=0).mean()

If we want to download data for multiple indicators, we need to specify them as a list. For example, let's choose Total Fertility Rate and GDP per Capita.

In [ ]:
df = wb.download(indicator=['SP.DYN.TFRT.IN','NY.GDP.PCAP.KD'], country=['US','CA','MX'], start=2008, end=2013)
df

In [ ]:
df.groupby(level=0).mean()

We can download data for a single year by setting the start and end dates to the same year. If we only want one country, we just name it.

In [ ]:
df = wb.download(indicator='NY.GDP.PCAP.KD', country='US', start=2013, end=2013)
df

To download the data for all countries, set the country attribute to 'all'.

In [ ]:
df = wb.download(indicator='SP.DYN.TFRT.IN', country='all', start=2013, end=2013)
df.tail()

Notice that selecting *all* countries also pulls indicators back for different regional groupings as well as countries.

## An example with some modeling

Suppose you want to compare Total Fertility Rate and GDP per Capita.

In [ ]:
wb.search('fertility rate').iloc[:,:2]

In [ ]:
wb.search('GDP per capita').iloc[:,:2]

Let's choose `SP.DYN.TFRT.IN` to compare to GDP (`NY.GDP.PCAP.KD`).

In [ ]:
ind = ['SP.DYN.TFRT.IN','NY.GDP.PCAP.KD']
df = wb.download(indicator=ind, country='all', start=2011, end=2011).dropna()
df.columns = ['fertility', 'gdp']
df.head()


In [ ]:
df.info()

Finally, we'll use the `statsmodels` package to assess the relationship between our two variables using ordinary least squares regression.

In [ ]:
import numpy as np
import statsmodels.formula.api as smf

results = smf.OLS(df['gdp'], df['fertility']).fit()
results.summary()


As might be expected, this model has a very low R-squared - fertility is NOT a good predictor of GDP.